## 1. Install Required Libraries

In [ ]:
!pip install boto3 fitz cohere faiss-cpu anthropic

## 2. Setup AWS S3 and Bedrock Clients

In [ ]:
import boto3
import faiss
import numpy as np
import fitz  # PyMuPDF
import cohere
from anthropic import Client as AnthropicClient

# Hardcoded AWS Credentials
aws_access_key = 'YOUR_AWS_ACCESS_KEY'
aws_secret_key = 'YOUR_AWS_SECRET_KEY'
region_name = 'us-east-1'

# Initialize S3 Client
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name=region_name
)

# Initialize Bedrock Client
bedrock = boto3.client(
    'bedrock',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name=region_name
)

# Initialize Cohere Client
cohere_client = cohere.Client(api_key="YOUR_COHERE_API_KEY")

# Initialize Anthropic Client
anthropic_client = AnthropicClient(api_key="YOUR_ANTHROPIC_API_KEY")


## 3. Extract Text, Images, and Hyperlinks from PDF

In [ ]:
def extract_text_and_links_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    links = []

    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text += page.get_text("text")
        links += page.get_links()
    
    return text, links


## 4. Recursive Character Splitter

In [ ]:
def recursive_split(text, chunk_size=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start = end - overlap
    return chunks


## 5. Generate Embeddings Using Cohere Embed English v3

In [ ]:
def generate_embeddings(text_chunks):
    embeddings = cohere_client.embed(texts=text_chunks, model="embed-english-v3.0")
    return np.array(embeddings.embeddings)


## 6. FAISS Vector Store and S3 Upload

In [ ]:
def create_faiss_index(embeddings, dimension=4096):
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)
    return index

def save_faiss_index_to_s3(index, bucket_name, s3_key):
    faiss.write_index(index, "faiss_index.index")
    s3_client.upload_file("faiss_index.index", bucket_name, s3_key)


## 7. Retrieve Relevant Chunks Using FAISS

In [ ]:
def retrieve_relevant_chunks(query_embedding, index, text_chunks, k=5):
    distances, indices = index.search(query_embedding, k)
    return [text_chunks[i] for i in indices[0]]


## 8. ReAct Agent Logic with Anthropic Sonnet

In [ ]:
def generate_response(query, relevant_chunks, chat_history):
    context = "\n".join(relevant_chunks)
    prompt = f"Given the following context: {context}\nAnswer the following question: {query}"
    
    # Include chat history
    full_prompt = "\n".join(chat_history + [prompt])

    response = anthropic_client.completion(
        model="sonnet-v1",  # Use the appropriate Sonnet model
        prompt=full_prompt,
        max_tokens=300
    )
    
    # Add the response to the chat history
    chat_history.append(f"User: {query}")
    chat_history.append(f"Sonnet: {response['completion']}")
    
    return response["completion"]


## 9. Memory Buffer for Chat History

In [ ]:
chat_history = []

def add_to_chat_history(query, response):
    chat_history.append({"query": query, "response": response})


## 10. Guardrails for LLM Responses

In [ ]:
def apply_guardrails(query):
    allowed_keywords = ["GenAI Hub"]
    if any(keyword.lower() in query.lower() for keyword in allowed_keywords):
        return True
    return False

def guarded_generate_response(query, relevant_chunks, chat_history):
    if not apply_guardrails(query):
        return "Sorry, I can only answer questions related to GenAI Hub."

    return generate_response(query, relevant_chunks, chat_history)


## 11. Evaluate Accuracy, Grounding, and Hallucination Detection

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_grounding(response, relevant_chunks, embedding_model):
    response_embedding = embedding_model.embed(texts=[response])
    chunk_embeddings = embedding_model.embed(texts=relevant_chunks)
    
    similarities = cosine_similarity(response_embedding, chunk_embeddings)
    return np.mean(similarities)

def check_hallucination(response, relevant_chunks):
    for chunk in relevant_chunks:
        if response in chunk:
            return False
    return True

def final_check(response, relevant_chunks, embedding_model):
    grounding_score = evaluate_grounding(response, relevant_chunks, embedding_model)
    hallucination_detected = check_hallucination(response, relevant_chunks)
    
    if hallucination_detected or grounding_score < 0.5:
        return "Hmm, I don't know"
    return response


## 12. Main Execution Logic in Jupyter Notebook

In [ ]:
def main(pdf_path, query, bucket_name, s3_key):
    # Step 1: Extract text and hyperlinks from the PDF
    text, links = extract_text_and_links_from_pdf(pdf_path)

    # Step 2: Split text using recursive splitter
    text_chunks = recursive_split(text)

    # Step 3: Generate embeddings using Cohere
    embeddings = generate_embeddings(text_chunks)

    # Step 4: Create FAISS index and upload to S3
    index = create_faiss_index(embeddings)
    save_faiss_index_to_s3(index, bucket_name, s3_key)

    # Step 5: Retrieve relevant chunks using FAISS
    query_embedding = generate_embeddings([query])[0].reshape(1, -1)
    relevant_chunks = retrieve_relevant_chunks(query_embedding, index, text_chunks)

    # Step 6: Generate response using ReAct agent and Anthropic
    response = guarded_generate_response(query, relevant_chunks, chat_history)

    # Step 7: Evaluate and detect hallucination
    final_response = final_check(response, relevant_chunks, cohere_client)
    
    # Display response
    print(final_response)

# Execute
pdf_path = "path_to_your_pdf_file.pdf"
query = "Your question here"
bucket_name = "your-s3-bucket"
s3_key = "faiss_index.index"

main(pdf_path, query, bucket_name, s3_key)
